In [44]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

stopwords = set(stopwords.words('english'))

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
from wordcloud import WordCloud
!pip install textblob
from textblob import TextBlob
!pip install textstat
import textstat

!pip install --upgrade Pillow

nltk.download('wordnet')
nltk.download('vader_lexicon')

sns.set_theme()

[nltk_data] Downloading package punkt to /Users/vikram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vikram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/vikram/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


[nltk_data] Downloading package wordnet to /Users/vikram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/vikram/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [45]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION 
reviews_df = pd.read_csv('/Users/vikram/Documents/COGS108/Group58_SP24/data/fake_reviews.csv')
reviews_df.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [46]:
# Formatting the names of the columns better and changing the ratings column from floats to integers
reviews_df = reviews_df.rename(columns = {'text_': 'text'})
reviews_df['category'] = reviews_df['category'].apply(lambda s: s[:-2].replace('_', ' '))
reviews_df['rating'] = reviews_df['rating'].astype(int)

In [47]:
# add a column with the stop words for each review removed and the punctuation for each review removed
reviews_df['text_no_stop'] = reviews_df['text'].apply(lambda s: ' '.join([token for token in word_tokenize(s.lower()) if token not in stopwords]))
reviews_df['text_no_punct'] = reviews_df['text'].apply(lambda s: s.lower().translate(str.maketrans('', '', string.punctuation)))
reviews_df.head()

,category,rating,label,text,text_no_stop,text_no_punct
0,Home and Kitchen,5,CG,"Love this! Well made, sturdy, and very comfor...","love ! well made , sturdy , comfortable . love...",love this well made sturdy and very comfortab...
1,Home and Kitchen,5,CG,"love it, a great upgrade from the original. I...","love , great upgrade original . 've mine coupl...",love it a great upgrade from the original ive...
2,Home and Kitchen,5,CG,This pillow saved my back. I love the look and...,pillow saved back . love look feel pillow .,this pillow saved my back i love the look and ...
3,Home and Kitchen,1,CG,"Missing information on how to use it, but it i...","missing information use , great product price !",missing information on how to use it but it is...
4,Home and Kitchen,5,CG,Very nice set. Good quality. We have had the s...,nice set . good quality . set two months,very nice set good quality we have had the set...


In [48]:
# Here we tokenize the words, making each word a unit, remove stop words (common words that don't differentiate the text well)
# Next we lemmatize the words, meaning break them down into their root meaning, thus words that come from the same root will be treated as the same
def preprocess_text(s):
    tokens = word_tokenize(s.lower())
    filtered_tokens = [token for token in tokens if token not in stopwords]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

In [49]:
# creates a new column called lemma_text that applies the preprocess_text function to the text_no_punct column
reviews_df['lemma_text'] = reviews_df['text_no_punct'].apply(preprocess_text)

In [50]:
# Assign the negative, positive, neutral, and composite sentiment values to columns
sent_analyzer = SentimentIntensityAnalyzer()
reviews_df['sentiment'] = reviews_df['lemma_text'].apply(lambda s: sent_analyzer.polarity_scores(s))
reviews_df['neg_sentiment'] = reviews_df.sentiment.apply(lambda dc: dc['neg'])
reviews_df['pos_sentiment'] = reviews_df.sentiment.apply(lambda dc: dc['pos'])
reviews_df['neu_sentiment'] = reviews_df.sentiment.apply(lambda dc: dc['neu'])
reviews_df['comp_sentiment'] = reviews_df.sentiment.apply(lambda dc: dc['compound'])

reviews_df.head()

,category,rating,label,text,text_no_stop,text_no_punct,lemma_text,sentiment,neg_sentiment,pos_sentiment,neu_sentiment,comp_sentiment
0,Home and Kitchen,5,CG,"Love this! Well made, sturdy, and very comfor...","love ! well made , sturdy , comfortable . love...",love this well made sturdy and very comfortab...,love well made sturdy comfortable love itvery ...,"{'neg': 0.0, 'neu': 0.15, 'pos': 0.85, 'compou...",0.000,0.850,0.150,0.9517
1,Home and Kitchen,5,CG,"love it, a great upgrade from the original. I...","love , great upgrade original . 've mine coupl...",love it a great upgrade from the original ive...,love great upgrade original ive mine couple year,"{'neg': 0.0, 'neu': 0.321, 'pos': 0.679, 'comp...",0.000,0.679,0.321,0.8910
2,Home and Kitchen,5,CG,This pillow saved my back. I love the look and...,pillow saved back . love look feel pillow .,this pillow saved my back i love the look and ...,pillow saved back love look feel pillow,"{'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'comp...",0.000,0.583,0.417,0.7906
3,Home and Kitchen,1,CG,"Missing information on how to use it, but it i...","missing information use , great product price !",missing information on how to use it but it is...,missing information use great product price,"{'neg': 0.214, 'neu': 0.388, 'pos': 0.398, 'co...",0.214,0.398,0.388,0.4404
4,Home and Kitchen,5,CG,Very nice set. Good quality. We have had the s...,nice set . good quality . set two months,very nice set good quality we have had the set...,nice set good quality set two month,"{'neg': 0.0, 'neu': 0.467, 'pos': 0.533, 'comp...",0.000,0.533,0.467,0.6908


In [51]:
# adds a column this the number of words in each review without punctuation
reviews_df['num_words'] = reviews_df['text_no_punct'].apply(lambda s: len(s.split(' ')))
# finds the average num of words in reviews grouped by whether the review is computer generated or not
review_len_per_group = reviews_df.groupby('label')['num_words'].mean()
review_len_per_group

label
CG    62.784032
OR    74.321231
Name: num_words, dtype: float64

In [52]:
def count_punc(text):
    count = 0
    for i in text:
        if i in string.punctuation:
            count += 1
    return count

reviews_df['num_punc_stand'] = reviews_df['text'].apply(count_punc) / reviews_df['num_words']
num_punc_per_group = reviews_df.groupby('label')['num_punc_stand'].mean()
num_punc_per_group

label
CG    0.150475
OR    0.160492
Name: num_punc_stand, dtype: float64

In [53]:
# categorizes reviews based on overall sentiment
def get_sentiment(review):
    analysis = TextBlob(review)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

# Apply sentiment analysis
reviews_df['sentiment_category'] = reviews_df['text'].apply(get_sentiment)

In [54]:
# Function to compute Flesch-Kincaid Grade Level
def compute_readability(review):
    return textstat.flesch_kincaid_grade(review)

# Apply the function to the 'review' column
reviews_df['readability'] = reviews_df['text'].apply(compute_readability)


In [55]:
## bag of words model 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews_df['text_no_punct'])

X_train, X_test, y_train, y_test = train_test_split(X, reviews_df['label'], test_size=0.2, random_state=42)
clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8606405341906764


In [56]:
# TFIDF model
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(reviews_df['text_no_punct'])

X_train, X_test, y_train, y_test = train_test_split(X, reviews_df['label'], test_size=0.2, random_state=42)

naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

y_pred = naive_bayes.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8712748856188945


In [70]:
tfidf_df = pd.DataFrame()
tfidf_df['label'] = reviews_df['label']
tfidf_df['text'] = reviews_df['lemma_text']

tfidf = TfidfVectorizer(sublinear_tf=True, analyzer='word', max_features=2000, tokenizer=word_tokenize)
tfidf_X = tfidf.fit_transform(tfidf_df['text']).toarray()

all_X = pd.concat([reviews_df[reviews_df.columns[-8:]], pd.DataFrame(tfidf_X)], axis = 1)

In [72]:
X_train

<32345x51256 sparse matrix of type '<class 'numpy.float64'>'
	with 1351704 stored elements in Compressed Sparse Row format>

In [84]:
all_X['sentiment_category'] = all_X['sentiment_category'].apply(lambda x: 0 if x == 'Negative' else 2 if x == 'Positive' else 1)

In [85]:
X_train, X_test, y_train, y_test = train_test_split(all_X, reviews_df['label'], test_size=0.2, random_state=42)

naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

y_pred = naive_bayes.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


/Users/vikram/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


ValueError: Negative values in data passed to MultinomialNB (input X)

array(['Negative', 'Positive', 'Neutral'], dtype=object)